# Car Price Regressor

In [ ]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

np.random.seed(42)

df = pd.read_csv("./data/car-sales.csv").sample(frac=1)

df.rename(
    columns={
        "Make": "make",
        "Colour": "color",
        "Odometer (KM)": "odometer",
        "Doors": "doors",
        "Price": "price",
    },
    inplace=True,
)

df.dropna(subset=["price"], inplace=True)

preprocessor = ColumnTransformer(
    transformers=[
        (
            "make_color",
            Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="constant", fill_value="NA")),
                    ("encoder", OneHotEncoder(handle_unknown="ignore")),
                ],
            ),
            ["make", "color"],
        ),
        (
            "doors",
            Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="most_frequent")),
                    ("encoder", OneHotEncoder(handle_unknown="ignore")),
                ],
            ),
            ["doors"],
        ),
        (
            "odometer",
            Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="mean")),
                ],
            ),
            ["odometer"],
        ),
    ],
    remainder="passthrough",
)

estimator = GridSearchCV(
    RandomForestRegressor(),
    param_grid={
        "n_estimators": [50, 100, 200],
        "min_samples_split": [2, 4, 8],
    },
    cv=5,
)

model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("estimator", estimator),
    ],
)

X = df.drop("price", axis=1)
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model.fit(X_train, y_train)

model.score(X_test, y_test)